In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
pd.options.display.max_columns=999

In [ ]:
path = 'Files/' 
età = pd.read_csv(path+'distribuzione_regionale_età.csv')
letalità = pd.read_csv(path+'letalità_Covid19.csv')
posti_ti = pd.read_csv(path+'Posti_letto_terapia_intensiva.csv')
regioni = pd.read_json('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-regioni.json')

In [ ]:
for index,row in letalità.iterrows():
    età.loc[(età['Età']>= row['min'])&(età['Età']<= row['max']),'fascia_età'] = row['fascia_età']
    età.loc[(età['Età']>= row['min'])&(età['Età']<= row['max']),'letalità'] = row['letalità']
    
età = età.groupby(['Territorio','fascia_età','letalità'], as_index = False)['Valore'].sum()
età['percentuale_età'] = età.groupby(['Territorio'])['Valore'].apply(lambda x: x / float(x.sum()))

In [ ]:
regione_casi = età.merge(regioni, left_on='Territorio', right_on='denominazione_regione', how= 'left')
regione_casi = regione_casi.merge(posti_ti, on='Territorio', how='left')
# calcoliamo la saturazione dei posti letto
regione_casi['TI_saturazione'] = regione_casi['terapia_intensiva'] / regione_casi['Posti letto TI']
# filtriamo le colonne che ci interessano
regione_casi = regione_casi[['Territorio','data','fascia_età','letalità','percentuale_età','TI_saturazione','totale_casi','deceduti','tamponi']]

In [ ]:
regione_casi.loc[regione_casi['Territorio']== 'Emilia Romagna'].head(10)